In [20]:
import pandas as pd
import numpy as np
import statistics
import csv
from sklearn import metrics
from xgboost import XGBClassifier
import random
import math
from sklearn.preprocessing import StandardScaler 
from sklearn.neural_network import MLPClassifier

In [18]:
#Extracting The Spatial Feature

def spatial_two (Node_class, Adj,n):
    F_vec=[]
    for i in range(n):
        #print("\rProcessing file {} ({}%)".format(i, 100*i//(n-1)), end='', flush=True)
        node_F=[]
        list_out=[]
        list_In=[]
        S_nbd_out=[]
        S_nbd_in=[]
        for j in range(n):
            if Adj[i][j]==1 and i!=j:
                list_out.append(Adj[j][j])
                for k in range(n):
                    if A[j][k]==1 and k!=j:
                        S_nbd_out.append(Adj[k][k])
            if Adj[j][i]==1 and i!=j:
                list_In.append(A[j][j])
                for k in range(n):
                    if Adj[k][j]==1 and k!=j:
                        S_nbd_in.append(Adj[k][k])

        for d in Node_class:
            count=0
            count_in=0
            for k in range(len(list_out)):
                if list_out[k]==d:
                    count=count+1
            node_F.append(count)
            for k in range(len(list_In)):
                if list_In[k]==d:
                    count_in=count_in+1
            node_F.append(count_in)
        for d in Node_class:
            count_S_out=0
            count_S_in=0
            for k in range(len(S_nbd_out)):
                if S_nbd_out[k]==d:
                    count_S_out=count_S_out+1
            node_F.append(count_S_out)
            for k in range(len(S_nbd_in)):
                if S_nbd_in[k]==d:
                    count_S_in=count_S_in+1
            node_F.append(count_S_in)
        F_vec.append(node_F)
    return F_vec

In [2]:
p_data=open("/Users/joshem/PhD Research/Data/Wisconsin/out1_graph_edges.txt")
myls=list(p_data)
mylist = [line.rstrip('\n') for line in myls]
mylist.pop(0)
print(mylist)

['63\t78', '63\t92', '27\t63', '27\t111', '110\t12', '98\t63', '98\t27', '98\t244', '98\t147', '98\t86', '98\t163', '98\t132', '98\t175', '98\t19', '98\t17', '98\t190', '98\t102', '98\t114', '98\t170', '98\t243', '98\t6', '98\t57', '98\t67', '98\t211', '98\t29', '98\t33', '98\t144', '98\t109', '98\t96', '98\t204', '98\t52', '98\t59', '98\t60', '98\t4', '98\t131', '98\t174', '98\t113', '98\t82', '98\t25', '98\t54', '98\t138', '98\t213', '98\t41', '98\t177', '98\t171', '98\t172', '98\t125', '98\t8', '98\t123', '98\t145', '98\t111', '98\t139', '98\t137', '98\t76', '98\t45', '98\t238', '98\t15', '98\t153', '98\t32', '98\t235', '98\t230', '98\t237', '98\t192', '98\t168', '98\t173', '98\t28', '98\t104', '98\t124', '98\t135', '98\t2', '98\t68', '98\t205', '98\t169', '98\t229', '98\t88', '98\t16', '98\t157', '98\t1', '98\t227', '98\t154', '98\t10', '98\t184', '98\t22', '98\t75', '98\t220', '98\t216', '98\t12', '98\t80', '98\t146', '98\t95', '98\t209', '98\t94', '98\t133', '98\t187', '98\t55', 

In [3]:
mynode=[]
i=0
for d in mylist:
    d=mylist[i].split("\t")
    mynode.append(int(d[0]))
    mynode.append(int(d[1]))
    i=i+1
#print(mynode)
node_ID1=np.unique(mynode)
node_ID=list(node_ID1)
print(node_ID)
n=len(node_ID)
print(n)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [4]:
A=np.zeros((n,n))
i=0
for d in mylist:
    d=mylist[i].split("\t")
    A[node_ID.index(int(d[0]))][node_ID.index(int(d[1]))]=1
    i=i+1;
print(" The adjacency Matrix")
print("=======================")
print(A)

 The adjacency Matrix
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [5]:
lst_ID=[]
lst_Fe=[]
lst_Class=[]
import csv
with open ("/Users/joshem/PhD Research/Data/Wisconsin/out1_node_feature_label.txt",newline='') as csvfile:
    data_reader=csv.reader(csvfile)
    next(data_reader)
    for row in data_reader:
        d=row[0].split('\t')
        lst_ID.append(int(d[0]))
        fev=[]
        fev=[d[1]]
        for j in range(1,len(row)-1):
            fev.append(int(row[j]))
        e=row[len(row)-1].split('\t')
        fev.append(e[0])
        lst_Class.append(int(e[1]))
        lst_Fe.append(fev)
        
print(lst_ID)
print(len(lst_ID))

print(len(lst_Fe[1]))
print(len(lst_Fe))
print(lst_Class)
print(len(lst_Class))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [6]:
feature_names = ["w_{}".format(ii) for ii in range(1703)]
Data=pd.DataFrame(lst_Fe,columns=feature_names)
Data.insert(loc=1703,column='Class',value=lst_Class)
Data.head()

,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,...,w_1694,w_1695,w_1696,w_1697,w_1698,w_1699,w_1700,w_1701,w_1702,Class
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,2
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,2
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,2
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [7]:
Class=lst_Class.copy()

In [8]:
#CLUSTERING DATA ACCORDING TO CLASS
Data_0=Data.loc[Data['Class'] == 0]
Data_1=Data.loc[Data['Class'] == 1]
Data_2=Data.loc[Data['Class'] == 2]
Data_3=Data.loc[Data['Class'] == 3]
Data_4=Data.loc[Data['Class'] == 4]
Data_1.head()

,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,...,w_1694,w_1695,w_1696,w_1697,w_1698,w_1699,w_1700,w_1701,w_1702,Class
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [9]:
# FINDING THE INCLUSIVE REPRESENTATIVE

Basis_0=[]
Basis_1=[]
Basis_2=[]
Basis_3=[]
Basis_4=[]
for i in range(1703):
    W="w_{}".format(i)
    Basis_0.append(max(Data_0[W].to_numpy()))
    Basis_1.append(max(Data_1[W].to_numpy()))
    Basis_2.append(max(Data_2[W].to_numpy()))
    Basis_3.append(max(Data_3[W].to_numpy()))
    Basis_4.append(max(Data_4[W].to_numpy()))
#W1=Data_0['w_0'].to_numpy()
#print(max(W1))
print(Basis_0)
count0=0
count1=0
for i in range(1703):
    if Basis_0[i]==1:
        count1=count1+1
    else:
        count0=count0+1
print(count0)
print(count1)

['0', 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0

In [11]:
# FINDING THE SELECTIVE REPRESENTATIVE

SBasis_0=[]
SBasis_1=[]
SBasis_2=[]
SBasis_3=[]
SBasis_4=[]
#for i in range(14):
 #   W="w_{}".format(i)
  #  print(list(Data_0[W].to_numpy()).count(1))
for i in range(1703):
    W="w_{}".format(i)
    if list(Data_0[W].to_numpy()).count(1)>=int(len(Data_0.index)*0.1):
        SBasis_0.append(1)
    else:
        SBasis_0.append(0) 
    if list(Data_1[W].to_numpy()).count(1)>=int(len(Data_1.index)*0.1):
        SBasis_1.append(1)
    else:
        SBasis_1.append(0)
    if list(Data_2[W].to_numpy()).count(1)>=int(len(Data_2.index)*0.1):
        SBasis_2.append(1)
    else:
        SBasis_2.append(0)
    if list(Data_3[W].to_numpy()).count(1)>=int(len(Data_3.index)*0.1):
        SBasis_3.append(1)
    else:
        SBasis_3.append(0)
    if list(Data_4[W].to_numpy()).count(1)>=int(len(Data_4.index)*0.1):
        SBasis_4.append(1)
    else:
        SBasis_4.append(0)

#W1=Data_0['w_0'].to_numpy()
#print(max(W1))
print(SBasis_3)
count0=0
count1=0
for i in range(1703):
    if SBasis_2[i]==1:
        count1=count1+1
    else:
        count0=count0+1
print(count0)
print(count1)

[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 

In [12]:
# FEATURE VECTOR USING INCLUSIVE REPRESENTATIVE

Fec=[]
for i in range(n):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    for j in range(1703):
        if f[j]==1 and Basis_0[j]==1:
            count=count+1;
        if f[j]==1 and Basis_1[j]==1:
            count1=count1+1;
        if f[j]==1 and Basis_2[j]==1:
            count2=count2+1;
        if f[j]==1 and Basis_3[j]==1:
            count3=count3+1;
        if f[j]==1 and Basis_4[j]==1:
            count4=count4+1;

    vec.append(count)
    vec.append(count1)
    vec.append(count2)
    vec.append(count3)
    vec.append(count4)

    #print(f)
    f.clear()
    Fec.append(vec)
print(Fec)

[[46, 72, 68, 59, 56], [39, 47, 49, 45, 45], [42, 48, 51, 49, 48], [27, 32, 33, 29, 27], [36, 64, 63, 58, 43], [81, 145, 129, 122, 103], [131, 242, 208, 176, 165], [35, 48, 48, 45, 37], [52, 69, 78, 67, 61], [85, 82, 83, 78, 72], [41, 43, 47, 45, 42], [52, 65, 70, 67, 60], [45, 53, 60, 55, 50], [50, 62, 74, 62, 58], [37, 42, 42, 42, 38], [50, 53, 63, 60, 56], [45, 47, 54, 45, 43], [36, 39, 41, 37, 34], [47, 61, 66, 66, 52], [22, 26, 26, 24, 22], [30, 48, 46, 39, 38], [79, 129, 115, 92, 88], [26, 30, 33, 28, 25], [44, 56, 67, 60, 56], [61, 97, 88, 76, 64], [82, 113, 116, 126, 99], [55, 94, 86, 83, 66], [81, 120, 126, 141, 120], [76, 142, 122, 112, 100], [46, 56, 69, 63, 59], [14, 14, 14, 14, 14], [75, 92, 107, 104, 92], [65, 72, 74, 65, 62], [28, 34, 36, 33, 28], [113, 205, 180, 177, 159], [138, 249, 220, 179, 168], [36, 74, 70, 68, 60], [83, 100, 114, 118, 98], [90, 152, 126, 112, 108], [36, 39, 41, 41, 38], [42, 49, 59, 44, 46], [55, 60, 67, 61, 57], [47, 84, 72, 69, 60], [42, 54, 70,

In [13]:
# FEATURE VECTOR USING SELECTIVE REPRESENTATIVE

SFec=[]
for i in range(n):
    Svec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0

    for j in range(1703):
        if f[j]==1 and SBasis_0[j]==1:
            count=count+1;
        if f[j]==1 and SBasis_1[j]==1:
            count1=count1+1;
        if f[j]==1 and SBasis_2[j]==1:
            count2=count2+1;
        if f[j]==1 and SBasis_3[j]==1:
            count3=count3+1;
        if f[j]==1 and SBasis_4[j]==1:
            count4=count4+1;

    Svec.append(count)
    Svec.append(count1)
    Svec.append(count2)
    Svec.append(count3)
    Svec.append(count4)

    #print(f)
    f.clear()
    SFec.append(Svec)
print(SFec)

[[46, 63, 40, 52, 44], [39, 38, 40, 44, 38], [42, 34, 40, 43, 42], [27, 27, 27, 26, 24], [36, 54, 36, 43, 35], [81, 105, 54, 81, 78], [131, 174, 63, 104, 115], [35, 35, 27, 36, 32], [52, 49, 48, 54, 49], [85, 55, 43, 60, 59], [41, 35, 39, 41, 39], [52, 42, 47, 59, 52], [45, 32, 39, 46, 41], [50, 43, 41, 55, 45], [37, 33, 37, 39, 35], [50, 40, 47, 56, 46], [45, 35, 37, 32, 37], [36, 30, 36, 32, 29], [47, 42, 41, 52, 43], [22, 21, 22, 21, 22], [30, 38, 25, 32, 30], [79, 103, 44, 66, 62], [26, 25, 28, 23, 22], [44, 37, 37, 56, 49], [61, 86, 38, 52, 45], [82, 68, 50, 97, 75], [55, 78, 39, 56, 53], [81, 53, 47, 118, 95], [76, 113, 45, 76, 70], [46, 40, 47, 61, 48], [14, 14, 14, 14, 14], [75, 56, 58, 92, 78], [65, 49, 49, 52, 51], [28, 29, 31, 28, 27], [113, 121, 58, 126, 123], [138, 182, 64, 105, 118], [36, 55, 31, 47, 44], [83, 56, 56, 105, 85], [90, 116, 43, 76, 82], [36, 34, 34, 37, 32], [42, 34, 36, 31, 35], [55, 43, 53, 55, 50], [47, 69, 40, 52, 47], [42, 32, 32, 42, 41], [75, 60, 45, 

In [14]:
z =np.array(SFec)
data2=pd.DataFrame({"S_0":z[:,0]})
for i in range(4):
    data2.insert(loc=i+1, column="S_{}".format(i+1), value=z[:,i+1])
#data2.insert(loc=7,column='Class',value=Data['subject'].to_numpy())
data2.head()

,S_0,S_1,S_2,S_3,S_4
0,46,63,40,52,44
1,39,38,40,44,38
2,42,34,40,43,42
3,27,27,27,26,24
4,36,54,36,43,35


In [15]:
y =np.array(Fec)
data1=pd.DataFrame({"In_0":y[:,0]})
for i in range(4):
    data1.insert(loc=i+1, column="In_{}".format(i+1), value=y[:,i+1])
#data1.insert(loc=5,column='Class',value=Data['Class'].to_numpy())
data1.head()

,In_0,In_1,In_2,In_3,In_4
0,46,72,68,59,56
1,39,47,49,45,45
2,42,48,51,49,48
3,27,32,33,29,27
4,36,64,63,58,43


In [16]:
Ac_fe=[]
acuracy=[]
Ac_domain2=[]
Ac_domain=[]

In [23]:
Ac_fe=[]
acuracy=[]
Ac_domain2=[]
Ac_domain=[]

for it in range(10):
    p=[]
    q=[]

    p=random.sample(range(n), math.ceil(n*0.2))
    q=[]
    for i in range(n):
        if i not in p:
            q.append(i)
    class_60=lst_Class.copy()
    for d in p:
        class_60[d]=5
    for i in range(n):
        A[i][i]=class_60[i]
    
    Node_class=[0,1,2,3,4]
    F_vec=spatial_two (Node_class, A,n)
    x =np.array(F_vec)
    k=20
    data=pd.DataFrame({"a_0":x[:,0]})
    data.insert(loc=1, column="b_0", value=x[:,1])
    for i in range(1,k-1,2):
        data.insert(loc=i+1, column="a_{}".format(int((i+1)/2)), value=x[:,i+1])
        data.insert(loc=i+2, column="b_{}".format(int((i+1)/2)), value=x[:,i+2])

    data.insert(loc=k,column='Class',value=Data['Class'].to_numpy())
    feature=["a_0","b_0"]
    for i in range(1,k-1,2):
        feature.append("a_{}".format(int((i+1)/2)))
        feature.append("b_{}".format(int((i+1)/2)))

    X=data[feature] # Features
    y=data['Class']  # Labels
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]
    scaler = StandardScaler()  
# Don't cheat - fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
# apply same transformation to test data
    X_test = scaler.transform(X_test)  
    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    
    Ac_domain2.append(metrics.accuracy_score(y_test, y_pred)*100)
    result = pd.concat([data2,data1,data], axis=1)
    k=20
    feature=["a_0","b_0"]
    for i in range(1,k-1,2):
        feature.append("a_{}".format(int((i+1)/2)))
        feature.append("b_{}".format(int((i+1)/2)))
    for i in range(4):
        feature.append("S_{}".format(i))
    for j in range(4):
        feature.append("In_{}".format(j))
    #print(feature)
    X=result[feature] # Features
    y=result['Class']  # Labels
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]

    scaler = StandardScaler()  
# Don't cheat - fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
# apply same transformation to test data
    X_test = scaler.transform(X_test)  
    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    
    acuracy.append(metrics.accuracy_score(y_test, y_pred)*100)
    feature=[]
    for i in range(4):
        feature.append("S_{}".format(i))
    for j in range(4):
        feature.append("In_{}".format(j))
    #print(feature)
    X=result[feature] # Features
    y=result['Class']  # Labels
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]
    scaler = StandardScaler()  
# Don't cheat - fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
# apply same transformation to test data
    X_test = scaler.transform(X_test)  
    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    Ac_fe.append(metrics.accuracy_score(y_test, y_pred)*100)

In [28]:

mean_value = statistics.mean(Ac_domain2)
std_dev = statistics.stdev(Ac_domain2)
print("Spatial only")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)

mean_value = statistics.mean(acuracy)
std_dev = statistics.stdev(acuracy)
print("Accuracy using both")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)
mean_value = statistics.mean(Ac_fe)
std_dev = statistics.stdev(Ac_fe)
print("Domain only")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)

Spatial only
Mean: 62.549019607843135
Standard Deviation: 7.304488141672424
Accuracy using both
Mean: 84.31372549019608
Standard Deviation: 5.468366186497227
Domain only
Mean: 88.43137254901961
Standard Deviation: 3.9702856335914873


In [27]:
print(acuracy)

[84.31372549019608, 82.35294117647058, 86.27450980392157, 72.54901960784314, 88.23529411764706, 78.43137254901961, 86.27450980392157, 86.27450980392157, 86.27450980392157, 92.15686274509804]


In [26]:
import warnings 